In [0]:
spark

sc = spark.sparkContext

In [0]:
from pyspark.sql import functions as F # This will load the class where spark sql functions are contained
from pyspark.sql import Row # this will let us manipulate rows with spark sql

In [0]:
users_dct = [
    {'id': 1, 'name': 'George', 'orders': [50.61, 31.32, 20.9]},
    {'id': 2, 'name': 'Hugues', 'orders': [133.8, 59.0, 40.03, 27.91]}
]
users_rdd = sc.parallelize(users_dct)
users_df = spark.createDataFrame(users_rdd.map(lambda x: Row(**x))) # this is called unpacking, 
# try this command with Row(x) and Row(*x) to understand what it does
users_df.show()

+---+------+--------------------+
| id|  name|              orders|
+---+------+--------------------+
|  1|George|[50.61, 31.32, 20.9]|
|  2|Hugues|[133.8, 59.0, 40....|
+---+------+--------------------+



In [0]:
# The .createDataFrame(...) method is able to infer the data schema by itself
users_df.printSchema()

root
 |-- id: long (nullable = true)
 |-- name: string (nullable = true)
 |-- orders: array (nullable = true)
 |    |-- element: double (containsNull = true)



In [0]:
from pyspark.sql.types import * # Import types to convert columns using spark sql

In [0]:
users_dct = [
    {'id': 1, 'name': 'George', 'orders': [50, 31, 20]},
    {'id': 2, 'name': 'Hugues', 'orders': [133, 59, 40, 27]}
]
users_rdd = sc.parallelize(users_dct)

# we create a variable schema as a list of StructField inside a StructType object
schema = StructType([
    StructField('id', IntegerType(), True), # the first column is of type Integer
    StructField('name', StringType(), True), # the second column is a String
    StructField('orders', ArrayType(IntegerType()), True) # the third column contains Array of Integer
])

users_df = spark.createDataFrame(users_rdd.map(lambda x: Row(**x)), schema=schema) # we feed the schema
# to the function using the appropriate argument
users_df.printSchema()
users_df.show()

root
 |-- id: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- orders: array (nullable = true)
 |    |-- element: integer (containsNull = true)

+---+------+-----------------+
| id|  name|           orders|
+---+------+-----------------+
|  1|George|     [50, 31, 20]|
|  2|Hugues|[133, 59, 40, 27]|
+---+------+-----------------+



In [0]:
users_df \
    .withColumn('orders_quantity', F.size('orders')) \
    .drop('orders') \
    .show()


+---+------+---------------+
| id|  name|orders_quantity|
+---+------+---------------+
|  1|George|              3|
|  2|Hugues|              4|
+---+------+---------------+



In [0]:
orders_df = users_df.withColumn('orders', F.explode('orders'))
orders_df.printSchema()
orders_df.show()

root
 |-- id: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- orders: integer (nullable = true)

+---+------+------+
| id|  name|orders|
+---+------+------+
|  1|George|    50|
|  1|George|    31|
|  1|George|    20|
|  2|Hugues|   133|
|  2|Hugues|    59|
|  2|Hugues|    40|
|  2|Hugues|    27|
+---+------+------+



In [0]:
orders_df.groupBy('id', 'name') \
    .mean('orders') \
    .show()

# here it's ok to just writethe column names, but don't forget that it's usually
# better to use the column objects instead to avoid errors 

+---+------+------------------+
| id|  name|       avg(orders)|
+---+------+------------------+
|  1|George|33.666666666666664|
|  2|Hugues|             64.75|
+---+------+------------------+



In [0]:
orders_df.groupBy('id', 'name') \
    .agg(F.collect_list('orders').alias('orders')) \
    .show()

+---+------+-----------------+
| id|  name|           orders|
+---+------+-----------------+
|  1|George|     [50, 31, 20]|
|  2|Hugues|[133, 59, 40, 27]|
+---+------+-----------------+



In [0]:
from pyspark.sql.types import *

In [0]:
users = [
    {'id': 1, 'name': 'George', 'orders': [
        {'id': 1, 'value': 55.1},
        {'id': 2, 'value': 78.31},
        {'id': 4, 'value': 52.13}
    ]},
    {'id': 2, 'name': 'Hughes', 'orders': [
        {'id': 3, 'value': 31.19},
        {'id': 5, 'value': 131.1}
    ]}
]
users_rdd = sc.parallelize(users)

schema = StructType([
    StructField('id', IntegerType(), True),
    StructField('name', StringType(), True),
    StructField('orders', ArrayType(
        StructType([
            StructField('id', IntegerType(), True),
            StructField('value', FloatType(), True)
        ])
    ), True)
])

users_df = spark.createDataFrame(users_rdd, schema=schema)
users_df.printSchema()
users_df.show()

# You'll see that the schema this time is a little deeper than before!

root
 |-- id: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- orders: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- id: integer (nullable = true)
 |    |    |-- value: float (nullable = true)

+---+------+--------------------+
| id|  name|              orders|
+---+------+--------------------+
|  1|George|[{1, 55.1}, {2, 7...|
|  2|Hughes|[{3, 31.19}, {5, ...|
+---+------+--------------------+



In [0]:
# Let's explode the orders column start unnesting the schema
orders_df = users_df.withColumn('orders', F.explode('orders'))
orders_df.printSchema()
orders_df.show()

root
 |-- id: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- orders: struct (nullable = true)
 |    |-- id: integer (nullable = true)
 |    |-- value: float (nullable = true)

+---+------+----------+
| id|  name|    orders|
+---+------+----------+
|  1|George| {1, 55.1}|
|  1|George|{2, 78.31}|
|  1|George|{4, 52.13}|
|  2|Hughes|{3, 31.19}|
|  2|Hughes|{5, 131.1}|
+---+------+----------+



In [0]:
orders_df \
    .withColumn('order_id', F.col('orders').getField('id')) \
    .show()

# F.col("col_name") returns the column object just like df.col_name or df["col_name"]

+---+------+----------+--------+
| id|  name|    orders|order_id|
+---+------+----------+--------+
|  1|George| {1, 55.1}|       1|
|  1|George|{2, 78.31}|       2|
|  1|George|{4, 52.13}|       4|
|  2|Hughes|{3, 31.19}|       3|
|  2|Hughes|{5, 131.1}|       5|
+---+------+----------+--------+



In [0]:
orders_df \
    .withColumn('order_id', F.col('orders.id')) \
    .show()

+---+------+----------+--------+
| id|  name|    orders|order_id|
+---+------+----------+--------+
|  1|George| {1, 55.1}|       1|
|  1|George|{2, 78.31}|       2|
|  1|George|{4, 52.13}|       4|
|  2|Hughes|{3, 31.19}|       3|
|  2|Hughes|{5, 131.1}|       5|
+---+------+----------+--------+



In [0]:
# Let's extract both the nested columns to get a flat schema
orders_df_flattened = orders_df \
    .withColumn('order_id', F.col('orders.id')) \
    .withColumn('order_value', F.col('orders.value')) \
    .drop('orders')
orders_df_flattened.show()

+---+------+--------+-----------+
| id|  name|order_id|order_value|
+---+------+--------+-----------+
|  1|George|       1|       55.1|
|  1|George|       2|      78.31|
|  1|George|       4|      52.13|
|  2|Hughes|       3|      31.19|
|  2|Hughes|       5|      131.1|
+---+------+--------+-----------+



In [0]:
# It is now possible to aggregate this table using goupBy and some aggregation function like .sum
orders_df_flattened \
    .groupBy('name') \
    .sum('order_value') \
    .orderBy('sum(order_value)') \
    .show()

+------+------------------+
|  name|  sum(order_value)|
+------+------------------+
|Hughes|162.29000663757324|
|George|185.53999710083008|
+------+------------------+



In [0]:
# Aliasing inline and descending sort
orders_df_flattened \
    .groupBy('name') \
    .agg(F.sum('order_value').alias('total_value')) \
    .orderBy(F.desc('total_value')) \
    .show()

+------+------------------+
|  name|       total_value|
+------+------------------+
|George|185.53999710083008|
|Hughes|162.29000663757324|
+------+------------------+



In [0]:
users = [
    {'id': 1, 'name': 'George', 'orders': [
        {'id': 1, 'items': [
            {'id': 1, 'category': 'shirt', 'price': 80, 'quantity': 4},
            {'id': 2, 'category': 'jeans', 'price': 130, 'quantity': 2}
        ]},
        {'id': 4, 'items': [
            {'id': 1, 'category': 'shirt', 'price': 80, 'quantity': 1},
            {'id': 3, 'category': 'shoes', 'price': 240, 'quantity': 1}
        ]}
    ]},
    {'id': 2, 'name': 'Hughes', 'orders': [
        {'id': 2, 'items': [
            {'id': 4, 'category': 'shorts', 'price': 120, 'quantity': 3},
            {'id': 1, 'category': 'shirt', 'price': 180, 'quantity': 2},
            {'id': 3, 'category': 'shoes', 'prices': 240, 'quantity': 1}
        ]},
        {'id': 3, 'items': [
            {'id': 5, 'category': 'suit', 'price': 2000, 'quantity': 1}
        ]}
    ]}
]
users_rdd = sc.parallelize(users)

schema = StructType([
    StructField('id', IntegerType(), True),
    StructField('name', StringType(), True),
    StructField('orders', ArrayType(
        StructType([
            StructField('id', IntegerType(), True),
            StructField('items', ArrayType(
                StructType([
                    StructField('id', IntegerType(), True),
                    StructField('category', StringType(), True),
                    StructField('price', IntegerType(), True),
                    StructField('quantity', IntegerType(), True)
                ])
            ))
        ])
    ), True)
])

users_df = spark.createDataFrame(users_rdd, schema=schema)
users_df.printSchema()
users_df.show()

# This schema is much deeper than the other two!

root
 |-- id: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- orders: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- id: integer (nullable = true)
 |    |    |-- items: array (nullable = true)
 |    |    |    |-- element: struct (containsNull = true)
 |    |    |    |    |-- id: integer (nullable = true)
 |    |    |    |    |-- category: string (nullable = true)
 |    |    |    |    |-- price: integer (nullable = true)
 |    |    |    |    |-- quantity: integer (nullable = true)

+---+------+--------------------+
| id|  name|              orders|
+---+------+--------------------+
|  1|George|[{1, [{1, shirt, ...|
|  2|Hughes|[{2, [{4, shorts,...|
+---+------+--------------------+



In [0]:
# We start by exploding the orders column, which where the nest resides
orders_df = users_df.withColumn('orders', F.explode('orders'))
orders_df.show()

+---+------+--------------------+
| id|  name|              orders|
+---+------+--------------------+
|  1|George|{1, [{1, shirt, 8...|
|  1|George|{4, [{1, shirt, 8...|
|  2|Hughes|{2, [{4, shorts, ...|
|  2|Hughes|{3, [{5, suit, 20...|
+---+------+--------------------+



In [0]:
items_df = (
    orders_df.withColumn('order_id', F.col('orders.id'))
    .withColumn('items', F.col('orders.items'))
    .drop('orders')
    .withColumnRenamed('name', 'user_name')
    .withColumnRenamed('id', 'user_id')
    .withColumn('items', F.explode('items'))
    .withColumn('item_id', F.col('items.id'))
    .withColumn('item_category', F.col('items.category'))
    .withColumn('item_price', F.col('items.price'))
    .withColumn('item_quantity', F.col('items.quantity'))
    .withColumn('total_price', F.col('item_price') * F.col('item_quantity'))
    .drop('items')
)
items_df.show()

+-------+---------+--------+-------+-------------+----------+-------------+-----------+
|user_id|user_name|order_id|item_id|item_category|item_price|item_quantity|total_price|
+-------+---------+--------+-------+-------------+----------+-------------+-----------+
|      1|   George|       1|      1|        shirt|        80|            4|        320|
|      1|   George|       1|      2|        jeans|       130|            2|        260|
|      1|   George|       4|      1|        shirt|        80|            1|         80|
|      1|   George|       4|      3|        shoes|       240|            1|        240|
|      2|   Hughes|       2|      4|       shorts|       120|            3|        360|
|      2|   Hughes|       2|      1|        shirt|       180|            2|        360|
|      2|   Hughes|       2|      3|        shoes|      null|            1|       null|
|      2|   Hughes|       3|      5|         suit|      2000|            1|       2000|
+-------+---------+--------+----

In [0]:
# Here we group the data by item category and calculate the sum
items_df \
    .groupBy('item_category') \
    .sum('item_quantity') \
    .orderBy(F.desc('sum(item_quantity)')) \
    .show()

+-------------+------------------+
|item_category|sum(item_quantity)|
+-------------+------------------+
|        shirt|                 7|
|       shorts|                 3|
|        shoes|                 2|
|        jeans|                 2|
|         suit|                 1|
+-------------+------------------+



In [0]:
items_df \
    .groupBy('item_category') \
    .agg(F.sum('item_quantity').alias('total_quantity')) \
    .orderBy(F.desc('total_quantity')) \
    .show()

+-------------+--------------+
|item_category|total_quantity|
+-------------+--------------+
|        shirt|             7|
|       shorts|             3|
|        shoes|             2|
|        jeans|             2|
|         suit|             1|
+-------------+--------------+



In [0]:
items_df \
    .groupBy('item_category') \
    .agg((F.sum('total_price') / F.sum('item_quantity')).alias('avg_sale')) \
    .orderBy(F.desc('avg_sale')) \
    .show()

+-------------+------------------+
|item_category|          avg_sale|
+-------------+------------------+
|         suit|            2000.0|
|        jeans|             130.0|
|        shoes|             120.0|
|       shorts|             120.0|
|        shirt|108.57142857142857|
+-------------+------------------+

